# Introduction
This notebook contains code for calling Genetic Report Card (GRC) alleles from a VCF file. The code uses the GT field of the VCF without any filtering, so is liekly to be very sensitive, at the possible risk of an elevated calling error rate. Note the code also makes use of GATK HaplotypeCaller phasing information (PGT and PID fields).

Following the running of the code, a few loci in a few samples are inspected as comparison with Olivo reads-based calling code highlighted 8 samples with different alleles called.

In [1]:
%run _standard_imports.ipynb

python 3.4.5 |Anaconda 2.2.0 (64-bit)| (default, Jul  2 2016, 17:47:47) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.1
scipy 0.18.0
pandas 0.19.0
numexpr 2.6.1
pysam 0.9.1.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
scratch_dir = "/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161118_GRC_from_VCF"
output_dir = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161118_GRC_from_VCF"
!mkdir -p {scratch_dir}
!mkdir -p {output_dir}

ref_fasta_fn = "%s/Pfalciparum.genome.fasta" % scratch_dir
vcf_file_format = "/nfs/team112_internal/production_files/Pf/6_0/vcf/SNP_INDEL_%s.combined.filtered.vcf.gz"

# See 20161115_run_Olivo_GRC.ipynb
grc_properties_fn = '/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161117_run_Olivo_GRC/grc/grc.properties'

grc_from_vcf_fn = "%s/Pf_6_GRC_from_vcf.xlsx" % output_dir

In [3]:
grc_from_vcf_fn

'/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161118_GRC_from_VCF/Pf_6_GRC_from_vcf.xlsx'

In [4]:
!cp /lustre/scratch116/malaria/pfalciparum/resources/Pfalciparum.genome.fasta {ref_fasta_fn}

In [5]:
def call_haplotypes(target_name='crt_72-76', chrom='Pf3D7_07_v3', start=403612, end=403626, strand='+',
    samples=None, positions_with_missingness=[403620, 403621], ref_fasta_fn=ref_fasta_fn, vcf_fn=None,
    verbose=True, show_genotypes=False
):
    
    from pyfasta import Fasta
    from Bio.Seq import Seq
    
    if vcf_fn is None:
        vcf_fn = vcf_file_format % chrom
    
    ref_sequence = Seq(Fasta(ref_fasta_fn)[chrom][start-1:end])
    vcf_reader = vcf.Reader(filename=vcf_fn)
    if samples is None:
        samples = vcf_reader.samples
    sample_sequences = collections.OrderedDict()
    sample_offsets = collections.OrderedDict()
    PID = collections.OrderedDict()
    non_phased_het_seen = collections.OrderedDict()
    genotype = collections.OrderedDict()
    num_missing = collections.OrderedDict()
    num_called = collections.OrderedDict()
    for sample in samples:
        sample_sequences[sample] = [ref_sequence.tomutable(), ref_sequence.tomutable()]
        sample_offsets[sample] = [0, 0]
        PID[sample] = ''
        non_phased_het_seen[sample] = False
        genotype[sample] = ''
        num_missing[sample] = 0
        num_called[sample] = 0

    for record in vcf_reader.fetch(chrom, start-1, end):
        if verbose:
            print(record, record.FILTER)
        for sample in samples:
            if show_genotypes:
                print(record.genotype(sample))
            GT = record.genotype(sample)['GT']
            POS = record.POS - start
            if GT == './.':
                if not record.POS in positions_with_missingness:
                    num_missing[sample] += 1
            else:
                num_called[sample] += 1
                if GT != '0/0':
                    if GT[0] != GT[2]: # heterozygous call
                        if 'PGT' in record.genotype(sample).data._fields:
                            is_unphased = (record.genotype(sample)['PGT'] is None)
                        else:
                            is_unphased = True
                        if non_phased_het_seen[sample]:
                            print("Sample %s unphased het followed by het" % sample)
                            genotype[sample] = 'X'
                        if is_unphased:
                            non_phased_het_seen[sample] = True
                        else:
                            if PID[sample] == '':
                                PID[sample] = record.genotype(sample)['PID']
                            else:
                                if record.genotype(sample)['PID'] != PID[sample]:
                                    print("Sample %s two PIDs" % sample)
                                    genotype[sample] = '?'
                            if record.genotype(sample)['PGT'] == '1|0':
                                GT = GT[2] + '/' + GT[0]

                    for i, sample_offset in enumerate(sample_offsets[sample]):
                        alleles = [record.REF] + record.ALT
                        GTint = int(GT[i*2])
                        REF_len = len(record.REF)
                        ALT_len = len(alleles[GTint])
                        if alleles[GTint] != '*':
                            sample_sequences[sample][i][POS+sample_offsets[sample][i]:(POS+sample_offsets[sample][i]+REF_len)] = alleles[GTint]
                            sample_offsets[sample][i] = sample_offset + (len(alleles[GTint]) - len(record.REF))

    for sample in samples:
        if(show_genotypes):
            print(sample_sequences[sample])
        if genotype[sample] == '':
            if num_missing[sample] > 0:
                genotype[sample] = '-'
            else:
                if sample_sequences[sample][0] == sample_sequences[sample][1]:
                    if strand == '+':
                        genotype[sample] = ( str(sample_sequences[sample][0].toseq().translate())
                                            if len(sample_sequences[sample][0]) % 3 == 0
                                            else '!' )
                    else:
                        genotype[sample] = ( str(sample_sequences[sample][0].toseq().reverse_complement().translate())
                                            if len(sample_sequences[sample][0]) % 3 == 0
                                            else '!' )
                else:
                    if strand == '+':
                        genotype[sample] = "%s,%s" % (
                            ( str(sample_sequences[sample][0].toseq().translate())
                             if len(sample_sequences[sample][0]) % 3 == 0
                             else '!' ),
                            ( str(sample_sequences[sample][1].toseq().translate())
                             if len(sample_sequences[sample][1]) % 3 == 0
                             else '!' )
                        )
                    else:
                        genotype[sample] = "%s,%s" % (
                            ( str(sample_sequences[sample][0].toseq().reverse_complement().translate())
                             if len(sample_sequences[sample][0]) % 3 == 0
                             else '!' ),
                            ( str(sample_sequences[sample][1].toseq().reverse_complement().translate())
                             if len(sample_sequences[sample][1]) % 3 == 0
                             else '!' )
                        )
                
    if strand == '+':
        ref_haplotype = str(ref_sequence.translate())
    else:
        ref_haplotype = str(ref_sequence.reverse_complement().translate())

    return(genotype, ref_haplotype)

In [6]:
def call_all_haplotypes(grc_properties_fn=grc_properties_fn):
    results = collections.OrderedDict()
    fi = open(grc_properties_fn, 'r')
    for line in fi:
        if line.startswith('grc.locus'):
            locus_name = line.split('.')[2]
            line_type = line.split('.')[3].split('=')[0]
            if line_type == 'region':
                chrom = line.split('.')[3].split('=')[1].split(':')[0]
            if line_type == 'targets':
                targets = line.split('=')[1].split(',')
                for target in targets:
                    target_name = target.split('@')[0]
                    target_coords = target.split('@')[1]
                    if target_coords.startswith('-'):
                        strand = '-'
                        target_coords = target_coords[1:]
                    else:
                        strand = '+'
                    start, end = target_coords.split('-')
                    print("\n", target_name)
                    genotype, ref_haplotype = call_haplotypes(target_name, chrom, int(start), int(end), strand)
                    locus_name = "%s[%s]" % (target_name, ref_haplotype)
                    results[locus_name] = genotype
    fi.close()
    
    tbl_results = (etl
        .fromcolumns(
            [list(results['crt_72-76[CVMNK]'].keys()), list(results['crt_72-76[CVMNK]'].values())],
            header=['Sample', 'crt_72-76[CVMNK]']
        )
    )
    for locus_name in list(results.keys())[1:]:
        print(locus_name)
        tbl_results = tbl_results.addcolumn(locus_name, results[locus_name].values())
    
    return(tbl_results)

In [7]:
tbl_all_results = call_all_haplotypes()


 crt_72-76
Record(CHROM=Pf3D7_07_v3, POS=403612, REF=T, ALT=[A]) []
Record(CHROM=Pf3D7_07_v3, POS=403613, REF=G, ALT=[A, C]) []
Record(CHROM=Pf3D7_07_v3, POS=403615, REF=G, ALT=[A]) []
Record(CHROM=Pf3D7_07_v3, POS=403618, REF=A, ALT=[AT]) []
Sample PH0060-C unphased het followed by het
Record(CHROM=Pf3D7_07_v3, POS=403620, REF=G, ALT=[T]) []
Sample PH0046-C two PIDs
Sample PH0242-C two PIDs
Sample PV0303-C two PIDs
Sample PV0323-C two PIDs
Sample QE0396-C two PIDs
Record(CHROM=Pf3D7_07_v3, POS=403621, REF=A, ALT=[G]) []
Sample PH0046-C two PIDs
Sample PH0242-C two PIDs
Sample PV0303-C two PIDs
Sample PV0323-C two PIDs
Sample QE0396-C two PIDs
Record(CHROM=Pf3D7_07_v3, POS=403622, REF=AT, ALT=[A]) []
Sample PH0060-C unphased het followed by het
Record(CHROM=Pf3D7_07_v3, POS=403623, REF=TA, ALT=[*, AA]) []
Sample PH0060-C unphased het followed by het
Record(CHROM=Pf3D7_07_v3, POS=403625, REF=A, ALT=[C]) []
Sample PN0008-C unphased het followed by het
Sample PN0054-C unphased het follow

In [8]:
tbl_all_results.toxlsx(grc_from_vcf_fn)

In [16]:
grc_from_vcf_fn

'/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20161118_GRC_from_VCF/Pf_6_GRC_from_vcf.xlsx'

# Manual inspection of calls and reads where discordant alleles

Note at this point I ran notebook 20161118_compare_Olivo_vs_VCF_GRC_Pf6.ipynb to look for discordant calls. The following cells are based on that analysis which showed discordant alleles in 8 samples. I also noticed that crt_97 had very few het calls when using VCF, so downloaded 3 bams where Olivo had called hets.

Other minor points investigated below are apparent frameshift mutations which turn out to be false, and two variants that fail filters, but for which inclusion or exclusion makes no difference to amino acids called.

## Discordant alleles

In [9]:
# Het call made at final SNP despite only one read
# Unclear which result (reads-based or vcf-based) really correct
call_haplotypes(samples=['PV0174-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=403612, REF=T, ALT=[A]) []
Call(sample=PV0174-C, CallData(GT=0/0, AD=[12, 0], BCS=[None, None, None, None], DP=12, GQ=30, PGT=None, PID=None, PL=[0, 30, 450]))
Record(CHROM=Pf3D7_07_v3, POS=403613, REF=G, ALT=[A, C]) []
Call(sample=PV0174-C, CallData(GT=0/0, AD=[12, 0, 0], BCS=[None, None, None, None], DP=12, GQ=30, PGT=None, PID=None, PL=[0, 30, 450, 30, 450, 450]))
Record(CHROM=Pf3D7_07_v3, POS=403615, REF=G, ALT=[A]) []
Call(sample=PV0174-C, CallData(GT=0/0, AD=[12, 0], BCS=[None, None, None, None], DP=12, GQ=30, PGT=None, PID=None, PL=[0, 30, 450]))
Record(CHROM=Pf3D7_07_v3, POS=403618, REF=A, ALT=[AT]) []
Call(sample=PV0174-C, CallData(GT=0/1, AD=[4, 6], BCS=[370, 73, 117, 414], DP=10, GQ=99, PGT=0|1, PID=403618_A_AT, PL=[179, 0, 187]))
Record(CHROM=Pf3D7_07_v3, POS=403620, REF=G, ALT=[T]) []
Call(sample=PV0174-C, CallData(GT=0/1, AD=[7, 3], BCS=[370, 73, 117, 414], DP=10, GQ=75, PGT=1|0, PID=403618_A_AT, PL=[75, 0, 282]))
Record(CHROM=Pf3D7_07_v3, PO

(OrderedDict([('PV0174-C', 'CVIDK,CVIET')]), 'CVMNK')

In [10]:
# Assume these are really the haplotypes Olivo finds here, but not enough evidence to call hets
call_haplotypes(samples=['QG0211-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=403612, REF=T, ALT=[A]) []
Call(sample=QG0211-C, CallData(GT=0/0, AD=[50, 4], BCS=[1833, 244, 606, 2070], DP=54, GQ=15, PGT=None, PID=None, PL=[0, 15, 1520]))
Record(CHROM=Pf3D7_07_v3, POS=403613, REF=G, ALT=[A, C]) []
Call(sample=QG0211-C, CallData(GT=0/0, AD=[53, 0, 0], BCS=[None, None, None, None], DP=53, GQ=99, PGT=None, PID=None, PL=[0, 120, 1800, 120, 1800, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403615, REF=G, ALT=[A]) []
Call(sample=QG0211-C, CallData(GT=0/0, AD=[53, 0], BCS=[None, None, None, None], DP=53, GQ=99, PGT=None, PID=None, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403618, REF=A, ALT=[AT]) []
Call(sample=QG0211-C, CallData(GT=0/0, AD=[52, 2], BCS=[1867, 276, 608, 2028], DP=54, GQ=85, PGT=0|1, PID=403618_A_AT, PL=[0, 85, 3710]))
Record(CHROM=Pf3D7_07_v3, POS=403620, REF=G, ALT=[T]) []
Call(sample=QG0211-C, CallData(GT=0/0, AD=[57, 0], BCS=[None, None, None, None], DP=57, GQ=99, PGT=None, PID=None, PL=[0, 104, 1715]))
Record(CHROM=Pf3

(OrderedDict([('QG0211-C', 'CVMNK,CVMNT')]), 'CVMNK')

In [11]:
# Get wrong call here becuase middle GT is 0/0. Would get correct call if we used PGT
call_haplotypes('dhps_436', 'Pf3D7_08_v3', 549681, 549683, '+', samples=['PH0919-C'], show_genotypes=True)

Record(CHROM=Pf3D7_08_v3, POS=549681, REF=T, ALT=[G, C]) []
Call(sample=PH0919-C, CallData(GT=0/1, AD=[2, 23, 0], BCS=[216, 156, 224, 327], DP=25, GQ=15, PGT=0|1, PID=549681_T_G, PL=[653, 0, 15, 659, 84, 743]))
Record(CHROM=Pf3D7_08_v3, POS=549682, REF=C, ALT=[T, G, A]) []
Call(sample=PH0919-C, CallData(GT=0/0, AD=[24, 2, 0, 0], BCS=[219, 162, 229, 343], DP=26, GQ=17, PGT=1|0, PID=549681_T_G, PL=[0, 17, 1025, 75, 1031, 1089, 75, 1031, 1089, 1089]))
Record(CHROM=Pf3D7_08_v3, POS=549683, REF=T, ALT=[C]) []
Call(sample=PH0919-C, CallData(GT=0/0, AD=[27, 0], BCS=[None, None, None, None], DP=27, GQ=61, PL=[0, 61, 887]))
[MutableSeq('Tct', Alphabet()), MutableSeq('Gct', Alphabet())]


(OrderedDict([('PH0919-C', 'S,A')]), 'S')

In [12]:
# Seems clear Olivo has it right here - final variant certainly looks like a het
call_haplotypes('dhps_540', 'Pf3D7_08_v3', 549993, 549995, '+', samples=['PD0009-01'], show_genotypes=True)

Record(CHROM=Pf3D7_08_v3, POS=549993, REF=A, ALT=[G, T]) []
Call(sample=PD0009-01, CallData(GT=0/1, AD=[8, 83, 0], BCS=[1586, 551, 396, 853], DP=91, GQ=71, PGT=0|1, PID=549993_A_G, PL=[2455, 0, 71, 2479, 319, 2798]))
Record(CHROM=Pf3D7_08_v3, POS=549994, REF=A, ALT=[T]) []
Call(sample=PD0009-01, CallData(GT=0/0, AD=[97, 0], BCS=[None, None, None, None], DP=97, GQ=99, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_08_v3, POS=549995, REF=A, ALT=[T, G]) []
Call(sample=PD0009-01, CallData(GT=0/0, AD=[86, 7, 0], BCS=[1608, 556, 396, 870], DP=93, GQ=20, PGT=1|0, PID=549993_A_G, PL=[0, 20, 3524, 259, 3545, 3784]))
[MutableSeq('Aaa', Alphabet()), MutableSeq('Gaa', Alphabet())]


(OrderedDict([('PD0009-01', 'K,E')]), 'K')

In [13]:
# Seems clear Olivo has it right here - final variant certainly looks like a het
call_haplotypes('dhps_540', 'Pf3D7_08_v3', 549993, 549995, '+', samples=['PH0566-C'], show_genotypes=True)

Record(CHROM=Pf3D7_08_v3, POS=549993, REF=A, ALT=[G, T]) []
Call(sample=PH0566-C, CallData(GT=0/1, AD=[5, 48, 0], BCS=[2474, 518, 616, 1503], DP=53, GQ=66, PGT=None, PID=None, PL=[1385, 0, 66, 1400, 210, 1610]))
Record(CHROM=Pf3D7_08_v3, POS=549994, REF=A, ALT=[T]) []
Call(sample=PH0566-C, CallData(GT=0/0, AD=[54, 0], BCS=[None, None, None, None], DP=54, GQ=99, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_08_v3, POS=549995, REF=A, ALT=[T, G]) []
Call(sample=PH0566-C, CallData(GT=0/0, AD=[49, 5, 0], BCS=[2516, 528, 627, 1531], DP=54, GQ=7, PGT=None, PID=None, PL=[0, 7, 2011, 147, 2026, 2166]))
[MutableSeq('Aaa', Alphabet()), MutableSeq('Gaa', Alphabet())]


(OrderedDict([('PH0566-C', 'K,E')]), 'K')

In [14]:
# Seems clear Olivo has it right here - final variant certainly looks like a het
call_haplotypes('dhps_540', 'Pf3D7_08_v3', 549993, 549995, '+', samples=['PH0718-C'], show_genotypes=True)

Record(CHROM=Pf3D7_08_v3, POS=549993, REF=A, ALT=[G, T]) []
Call(sample=PH0718-C, CallData(GT=0/1, AD=[3, 40, 0], BCS=[1964, 417, 501, 1231], DP=43, GQ=6, PGT=None, PID=None, PL=[1267, 0, 6, 1276, 126, 1402]))
Record(CHROM=Pf3D7_08_v3, POS=549994, REF=A, ALT=[T]) []
Call(sample=PH0718-C, CallData(GT=0/0, AD=[45, 0], BCS=[None, None, None, None], DP=45, GQ=99, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_08_v3, POS=549995, REF=A, ALT=[T, G]) []
Call(sample=PH0718-C, CallData(GT=0/0, AD=[41, 3, 0], BCS=[1996, 422, 507, 1257], DP=44, GQ=30, PGT=None, PID=None, PL=[0, 30, 1700, 123, 1709, 1802]))
[MutableSeq('Aaa', Alphabet()), MutableSeq('Gaa', Alphabet())]


(OrderedDict([('PH0718-C', 'K,E')]), 'K')

In [15]:
# Seems clear Olivo has it right here - final variant certainly looks like a het
call_haplotypes('dhps_540', 'Pf3D7_08_v3', 549993, 549995, '+', samples=['PH0905-C'], show_genotypes=True)

Record(CHROM=Pf3D7_08_v3, POS=549993, REF=A, ALT=[G, T]) []
Call(sample=PH0905-C, CallData(GT=0/1, AD=[4, 46, 0], BCS=[938, 330, 235, 507], DP=50, GQ=14, PGT=0|1, PID=549993_A_G, PL=[1399, 0, 14, 1411, 151, 1562]))
Record(CHROM=Pf3D7_08_v3, POS=549994, REF=A, ALT=[T]) []
Call(sample=PH0905-C, CallData(GT=0/0, AD=[50, 0], BCS=[None, None, None, None], DP=50, GQ=99, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_08_v3, POS=549995, REF=A, ALT=[T, G]) []
Call(sample=PH0905-C, CallData(GT=0/0, AD=[46, 2, 0], BCS=[918, 322, 227, 499], DP=48, GQ=81, PGT=1|0, PID=549993_A_G, PL=[0, 81, 2003, 144, 2009, 2072]))
[MutableSeq('Aaa', Alphabet()), MutableSeq('Gaa', Alphabet())]


(OrderedDict([('PH0905-C', 'K,E')]), 'K')

In [76]:
# Seems clear Olivo has it right here - final variant certainly looks like a het
call_haplotypes('dhps_540', 'Pf3D7_08_v3', 549993, 549995, '+', samples=['PH1422-C'], show_genotypes=True)

Record(CHROM=Pf3D7_08_v3, POS=549993, REF=A, ALT=[G, T]) []
Call(sample=PH1422-C, CallData(GT=0/1, AD=[3, 41, 0], BCS=[817, 283, 201, 439], DP=44, GQ=4, PGT=0|1, PID=549993_A_G, PL=[1212, 0, 4, 1221, 126, 1347]))
Record(CHROM=Pf3D7_08_v3, POS=549994, REF=A, ALT=[T]) []
Call(sample=PH1422-C, CallData(GT=0/0, AD=[46, 0], BCS=[None, None, None, None], DP=46, GQ=99, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_08_v3, POS=549995, REF=A, ALT=[T, G]) []
Call(sample=PH1422-C, CallData(GT=0/0, AD=[42, 3, 0], BCS=[828, 286, 202, 446], DP=45, GQ=24, PGT=1|0, PID=549993_A_G, PL=[0, 24, 1741, 126, 1750, 1852]))
[MutableSeq('Aaa', Alphabet()), MutableSeq('Gaa', Alphabet())]


(OrderedDict([('PH1422-C', 'K,E')]), 'K')

## Missed het calls in VCF
In each of these three cases I manually inspected the bam in IGV

In [86]:
# Sanity checking crt_97 that has very few het calls. This one was het by reads which said all reads aligned
# Inspection of bam shows 1 read with T at 403688, with good mapping and base quality - why is this not shown in AD?
call_haplotypes('crt_97', 'Pf3D7_07_v3', 403687, 403689, '+', samples=['PA0093-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=403687, REF=C, ALT=[T]) []
Call(sample=PA0093-C, CallData(GT=0/0, AD=[41, 0], BCS=[None, None, None, None], DP=41, GQ=99, PL=[0, 99, 1485]))
Record(CHROM=Pf3D7_07_v3, POS=403688, REF=A, ALT=[T]) []
Call(sample=PA0093-C, CallData(GT=0/0, AD=[41, 0], BCS=[None, None, None, None], DP=41, GQ=99, PGT=None, PID=None, PL=[0, 99, 1485]))
Record(CHROM=Pf3D7_07_v3, POS=403689, REF=C, ALT=[A, T]) []
Call(sample=PA0093-C, CallData(GT=0/0, AD=[41, 0, 0], BCS=[None, None, None, None], DP=41, GQ=99, PGT=None, PID=None, PL=[0, 99, 1485, 99, 1485, 1485]))
[MutableSeq('cac', Alphabet()), MutableSeq('cac', Alphabet())]


(OrderedDict([('PA0093-C', 'H')]), 'H')

In [89]:
!grep PA0093 /nfs/users/nfs_r/rp7/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/9/7/4/2/36984/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PA0093-C


In [87]:
# Sanity checking crt_97 that has very few het calls. This one was het by reads which said all reads aligned
# Inspection of bam shows 2 reads with T at 403688, with good mapping and base quality - why are these not shown in AD?
# Lower GQ of 403688 seems to suggest that these two reads have been taken into account
call_haplotypes('crt_97', 'Pf3D7_07_v3', 403687, 403689, '+', samples=['PH0905-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=403687, REF=C, ALT=[T]) []
Call(sample=PH0905-C, CallData(GT=0/0, AD=[35, 0], BCS=[None, None, None, None], DP=35, GQ=96, PL=[0, 96, 1440]))
Record(CHROM=Pf3D7_07_v3, POS=403688, REF=A, ALT=[T]) []
Call(sample=PH0905-C, CallData(GT=0/0, AD=[35, 0], BCS=[None, None, None, None], DP=35, GQ=31, PGT=None, PID=None, PL=[0, 31, 1110]))
Record(CHROM=Pf3D7_07_v3, POS=403689, REF=C, ALT=[A, T]) []
Call(sample=PH0905-C, CallData(GT=0/0, AD=[34, 0, 0], BCS=[None, None, None, None], DP=34, GQ=93, PGT=None, PID=None, PL=[0, 93, 1395, 93, 1395, 1395]))
[MutableSeq('cac', Alphabet()), MutableSeq('cac', Alphabet())]


(OrderedDict([('PH0905-C', 'H')]), 'H')

In [90]:
!grep PH0905 /nfs/users/nfs_r/rp7/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/4/e/d/3/41496/4_bam_mark_duplicates_v2/pe.1.markdup.bam	PH0905-C


In [88]:
# Sanity checking crt_97 that has very few het calls. This one was het by reads which said all reads aligned
# Inspection of bam shows 6 reads with TT at 403688/403689, with good mapping and base quality - why are these not shown in AD?
# Zero GQ of 403688 suggests that this was borderline het/hom call
call_haplotypes('crt_97', 'Pf3D7_07_v3', 403687, 403689, '+', samples=['QG0155-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=403687, REF=C, ALT=[T]) []
Call(sample=QG0155-C, CallData(GT=0/0, AD=[67, 0], BCS=[None, None, None, None], DP=67, GQ=99, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403688, REF=A, ALT=[T]) []
Call(sample=QG0155-C, CallData(GT=0/0, AD=[74, 0], BCS=[None, None, None, None], DP=74, GQ=0, PGT=None, PID=None, PL=[0, 0, 2147]))
Record(CHROM=Pf3D7_07_v3, POS=403689, REF=C, ALT=[A, T]) []
Call(sample=QG0155-C, CallData(GT=0/0, AD=[71, 0, 0], BCS=[None, None, None, None], DP=71, GQ=33, PGT=None, PID=None, PL=[0, 33, 2042, 33, 2042, 2042]))
[MutableSeq('cac', Alphabet()), MutableSeq('cac', Alphabet())]


(OrderedDict([('QG0155-C', 'H')]), 'H')

In [91]:
!grep QG0155 /nfs/users/nfs_r/rp7/pf_60_mergelanes.txt

/lustre/scratch116/malaria/pfalciparum/output/0/1/4/3/44018/4_bam_mark_duplicates_v2/pe.1.markdup.bam	QG0155-C


## Samples with apparent frameshift mutations
Two samples (PA0490-C and QV0090-C) had apparent frameshift mutations according to the VCF so warranted further inspection. In both cases, it seems a het call was wrongly called as hom, and hence frameshifts are not real.


In [18]:
# PA0490-C has hom alt call (4 ref, 134 alt reads) at 403618 insertion and het call (5 ref, 137 alt reads)
# at 403622 deletion. The results below suggest both calls should have been het, and hence no frameshift
call_haplotypes(samples=['PA0490-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=403612, REF=T, ALT=[A]) []
Call(sample=PA0490-C, CallData(GT=0/0, AD=[91, 0], BCS=[None, None, None, None], DP=91, GQ=99, PGT=None, PID=None, PL=[0, 102, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403613, REF=G, ALT=[A, C]) []
Call(sample=PA0490-C, CallData(GT=0/0, AD=[91, 0, 0], BCS=[None, None, None, None], DP=91, GQ=99, PGT=None, PID=None, PL=[0, 102, 1800, 102, 1800, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403615, REF=G, ALT=[A]) []
Call(sample=PA0490-C, CallData(GT=0/0, AD=[91, 0], BCS=[None, None, None, None], DP=91, GQ=99, PGT=None, PID=None, PL=[0, 102, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403618, REF=A, ALT=[AT]) []
Call(sample=PA0490-C, CallData(GT=1/1, AD=[4, 134], BCS=[5037, 1027, 1619, 5609], DP=138, GQ=25, PGT=1|1, PID=403618_A_AT, PL=[5667, 25, 0]))
Record(CHROM=Pf3D7_07_v3, POS=403620, REF=G, ALT=[T]) []
Call(sample=PA0490-C, CallData(GT=./., AD=[139, 0], BCS=[None, None, None, None], DP=139, GQ=None, PGT=None, PID=None, PL=[0, 0, 0]))
Record(CHR

(OrderedDict([('PA0490-C', '!,CVIET')]), 'CVMNK')

In [19]:
call_haplotypes(samples=['QV0090-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=403612, REF=T, ALT=[A]) []
Call(sample=QV0090-C, CallData(GT=0/0, AD=[64, 0], BCS=[None, None, None, None], DP=64, GQ=99, PGT=None, PID=None, PL=[0, 112, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403613, REF=G, ALT=[A, C]) []
Call(sample=QV0090-C, CallData(GT=0/0, AD=[64, 0, 0], BCS=[None, None, None, None], DP=64, GQ=99, PGT=None, PID=None, PL=[0, 112, 1800, 112, 1800, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403615, REF=G, ALT=[A]) []
Call(sample=QV0090-C, CallData(GT=0/0, AD=[64, 0], BCS=[None, None, None, None], DP=64, GQ=99, PGT=None, PID=None, PL=[0, 112, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=403618, REF=A, ALT=[AT]) []
Call(sample=QV0090-C, CallData(GT=0/1, AD=[3, 104], BCS=[3902, 722, 1251, 4330], DP=107, GQ=8, PGT=0|1, PID=403618_A_AT, PL=[4436, 0, 8]))
Record(CHROM=Pf3D7_07_v3, POS=403620, REF=G, ALT=[T]) []
Call(sample=QV0090-C, CallData(GT=./., AD=[110, 0], BCS=[None, None, None, None], DP=110, GQ=None, PGT=None, PID=None, PL=[0, 0, 0]))
Record(CHROM=

(OrderedDict([('QV0090-C', '!,CVIET')]), 'CVMNK')

## Effect of variants that failed filters
Two variants(at Pf3D7_07_v3:404836 and Pf3D7_07_v3:405559) failed variant filters. The following shows that in each case there was only one sample with a heterozygous call at the variant, and the position was the last in the codon and the different alleles made no change in the amino acid.


In [26]:
def non_ref_sample(chrom='Pf3D7_07_v3', pos=404838):
    vcf_reader = vcf.Reader(filename=vcf_file_format % chrom)
    samples = vcf_reader.samples
    for record in vcf_reader.fetch(chrom, pos-1, pos):
        print(record, record.FILTER)
        for sample in samples:
            GT = record.genotype(sample)['GT']
            if not GT in ['0/0', './.']:
                print(sample, record.genotype(sample))


In [27]:
non_ref_sample()

Record(CHROM=Pf3D7_07_v3, POS=404838, REF=A, ALT=[G]) ['Low_VQSLOD']
PF0671-C Call(sample=PF0671-C, CallData(GT=0/1, AD=[44, 22], BCS=[2566, 1015, 648, 2240], DP=66, GQ=99, PGT=None, PID=None, PL=[525, 0, 1348]))


In [29]:
call_haplotypes('crt_271', 'Pf3D7_07_v3', 404836, 404838, '+', samples=['PF0671-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=404836, REF=C, ALT=[G]) []
Call(sample=PF0671-C, CallData(GT=0/0, AD=[60, 0], BCS=[None, None, None, None], DP=60, GQ=99, PGT=None, PID=None, PL=[0, 120, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=404838, REF=A, ALT=[G]) ['Low_VQSLOD']
Call(sample=PF0671-C, CallData(GT=0/1, AD=[44, 22], BCS=[2566, 1015, 648, 2240], DP=66, GQ=99, PGT=None, PID=None, PL=[525, 0, 1348]))
[MutableSeq('caA', Alphabet()), MutableSeq('caG', Alphabet())]


(OrderedDict([('PF0671-C', 'Q,Q')]), 'Q')

In [28]:
non_ref_sample(chrom='Pf3D7_07_v3', pos=405559)

Record(CHROM=Pf3D7_07_v3, POS=405559, REF=C, ALT=[A]) ['Low_VQSLOD']
PF0844-C Call(sample=PF0844-C, CallData(GT=0/1, AD=[25, 61], BCS=[2695, 1261, 913, 3449], DP=86, GQ=99, PGT=None, PID=None, PL=[1931, 0, 587]))


In [30]:
call_haplotypes('crt_342', 'Pf3D7_07_v3', 405557, 405559, '+', samples=['PF0844-C'], show_genotypes=True)

Record(CHROM=Pf3D7_07_v3, POS=405557, REF=A, ALT=[G]) []
Call(sample=PF0844-C, CallData(GT=0/0, AD=[66, 0], BCS=[None, None, None, None], DP=66, GQ=99, PL=[0, 104, 1800]))
Record(CHROM=Pf3D7_07_v3, POS=405559, REF=C, ALT=[A]) ['Low_VQSLOD']
Call(sample=PF0844-C, CallData(GT=0/1, AD=[25, 61], BCS=[2695, 1261, 913, 3449], DP=86, GQ=99, PGT=None, PID=None, PL=[1931, 0, 587]))
[MutableSeq('acC', Alphabet()), MutableSeq('acA', Alphabet())]


(OrderedDict([('PF0844-C', 'T,T')]), 'T')